In [9]:
from sentence_transformers import SentenceTransformer, util

#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-distilbert-cos-v1')

In [10]:
query = "I just discovered the course. Can I still join it?"
docs = [""]

#Encode query and documents
query_emb = model.encode(query)
v = query_emb
doc_emb = model.encode(docs)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
#print(scores)

print ('Q1- What''s the first value of the resulting vector?')
print(query_emb[0])


Q1- Whats the first value of the resulting vector?
0.078222655


In [12]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [11]:
#filter
fitered_ml_docs = []
for doc in documents:
    if doc['course'] == 'machine-learning-zoomcamp':        
        fitered_ml_docs.append(doc)



In [13]:
#get embeddings_fitered_ml_docs
embeddings_fitered_ml_docs = []
for doc in fitered_ml_docs:
    qa_text = f"{doc['question']} {doc['text']}"
    embeddings_fitered_ml_docs.append(model.encode(qa_text))

In [14]:
#convert with numpy
import numpy as np
x=[]
x = np.array(embeddings_fitered_ml_docs)

print ('Q2- What''s the shape of X? (X.shape)')
print(x.shape)

Q2- Whats the shape of X? (X.shape)
(375, 768)


In [15]:
scores = x.dot(v)
max = np.max(scores)
print ('Q3 - What''s the highest score in the results?')
print(max)


Q3 - Whats the highest score in the results?
0.6506573


In [16]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


In [17]:
# np_emb_filtered_ml_docs = np.array(embeddings_fitered_ml_docs)
# banana= VectorSearchEngine(fitered_ml_docs, np_emb_filtered_ml_docs)
# maca = banana.search(query_emb, num_results=5)
# maca

In [45]:
import pandas as pd
from tqdm.auto import tqdm

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
gt_question_embeddings = []


In [46]:
#crate an embeddings array with all the questions from gt
for doc in tqdm(ground_truth):
    q = doc['question']
    q_v = model.encode(q)
    doc['vquestion']=q_v
    gt_question_embeddings.append(q_v)

100%|██████████| 1830/1830 [01:17<00:00, 23.51it/s]


In [47]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872',
 'vquestion': array([ 9.80187729e-02, -3.25485389e-03,  5.20954914e-02, -7.79575342e-03,
         8.67941231e-02,  3.88571173e-02,  4.64866422e-02,  3.89426425e-02,
         5.92647819e-03,  1.11361872e-02, -2.46879682e-02, -4.95579652e-02,
         1.01311065e-01, -2.28015445e-02,  2.11680271e-02, -1.74905304e-02,
         4.09106165e-02, -2.01090407e-02, -1.14281271e-02, -4.24045064e-02,
        -1.00629292e-02,  1.51091330e-02,  2.56886631e-02,  1.80889182e-02,
        -2.73963828e-02,  6.96142688e-02,  5.53158522e-02,  2.74974816e-02,
        -3.61659601e-02, -2.12623216e-02,  1.63038541e-02,  2.62813251e-02,
         3.82378139e-02,  6.22039335e-03,  2.72464268e-02, -3.14834900e-03,
        -3.23529094e-02, -1.93135384e-02,  2.69396175e-02,  5.85127063e-03,
        -4.19716612e-02,  1.15671484e-02,  4.71097231e-02, -1.33657095e-03,
         1.75673831e-02, -4.904909

In [48]:
import numpy as np
#convert to nparray (needed for the search)
np_queston_emb=[]
np_queston_emb = np.array(gt_question_embeddings)

In [49]:
#pass the original documents and the emb_questions nparray 
search_engine = VectorSearchEngine(ground_truth, np_queston_emb)

In [51]:

def hit_rate(relevance_total):
    cnt = 0
    for line in relevance_total:
        if True in line:
            cnt = cnt + 1
    return cnt / len(relevance_total)


def evaluateVSE(ground_truth, se):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results =  se(q['vquestion'], 5)
        relevance = [d['document'] == doc_id for d in results]
        relevance_total.append(relevance)
    return {
        'hit_rate': hit_rate(relevance_total)
    }

hitrate = evaluateVSE(ground_truth, search_engine.search)
print ('Q4 - Now use the code from the module to calculate the hitrate of VectorSearchEngine with num_results=5./nWhat did you get?')
print(hitrate['hit_rate'])


100%|██████████| 1830/1830 [00:00<00:00, 4195.63it/s]

Q4 - Now use the code from the module to calculate the hitrate of VectorSearchEngine with num_results=5./nWhat did you get?
0.9972677595628415


In [52]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [ ]:
##JSON
# base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
# relative_url = '03-vector-search/eval/documents-with-ids.json'
# docs_url = f'{base_url}/{relative_url}?raw=1'
# docs_response = requests.get(docs_url)
# documents = docs_response.json()

In [31]:
#CSV
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [32]:
for doc in tqdm(ground_truth):
    question = doc['question']
    # text = doc['text']
    # q_a = question + ' ' + text

    doc['vquestion'] = model.encode(question)
    # doc['vtext'] = model.encode(text)
    # doc['vqa'] = model.encode(q_a)



100%|██████████| 1830/1830 [01:19<00:00, 23.15it/s]


In [53]:
#es_client = Elasticsearch('http://localhost:9200')
es_client = Elasticsearch('http://localhost:9200', max_retries=10, retry_on_timeout=True) 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "vquestion": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "vtext": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "vqa": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [54]:
for doc in tqdm(ground_truth):
    es_client.index(index=index_name,document= doc)

100%|██████████| 1830/1830 [00:43<00:00, 41.92it/s]


In [ ]:

# #simple ES query POC
# q = ''
# v_q = query_emb

# queryParams = {
#      "field": "vquestion",
#      "query_vector": v_q,
#      "k": 5,
#      "num_candidates": 1000
# }


# es_result = es_client.search(
#     index=index_name, 
#     knn=queryParams,
#     source=["question", "course", "document"]
# )

# print('Total:', len(es_result['hits']['hits']))
# es_result['hits']['hits']

In [55]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'vquestion') + 1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["document", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["question", "course", "document"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs



In [56]:
results = elastic_search_knn('vquestion',query_emb, 'machine-learning-zoomcamp')
print ('Q5 - What''s the ID of the document with the highest score?')
print(results[0]['document'])


Q5 - Whats the ID of the document with the highest score?
ee58a693


In [41]:
def es_search(q):
    #question=q['question']
    course = q['course']
    #v_q = model.encode(question)

    return elastic_search_knn('vquestion', q['vquestion'], course)


def evaluate2(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results =  search_function(q)
        relevance = [d['document'] == doc_id for d in results]
        relevance_total.append(relevance)
    return {
        'hit_rate': hit_rate(relevance_total)
    }

evaluate2(ground_truth, es_search)

100%|██████████| 1830/1830 [01:35<00:00, 19.17it/s]


{'hit_rate': 0.9972677595628415}